In [1]:
import copy

In [2]:
copy.deepcopy(None)

In [3]:
from mlscorecheck.aggregated import (Fold, Dataset, Experiment,
                                        generate_dataset_specification,
                                        generate_experiment_specification)
from mlscorecheck.aggregated import solve
from mlscorecheck.experiments import dataset_statistics

In [4]:
experiment = Experiment(**generate_experiment_specification())

2023-08-25 10:31:10,049:INFO:a random identifier for the fold has been generated zomphmftrklmfnaf
2023-08-25 10:31:10,050:INFO:a random identifier for the fold has been generated jylyugpjnbnyrghy
2023-08-25 10:31:10,052:INFO:a random identifier for the fold has been generated bsckwaivackaehep
2023-08-25 10:31:10,052:INFO:a random identifier for the fold has been generated ruxbudqrgcuyrkqu
2023-08-25 10:31:10,053:INFO:a random identifier for the fold has been generated xboljkazwtytcaht
2023-08-25 10:31:10,055:INFO:a random identifier for the fold has been generated hisowqyyqdlqbspg
2023-08-25 10:31:10,055:INFO:a random identifier for the fold has been generated ximidhkogxlygcss
2023-08-25 10:31:10,056:INFO:a random identifier for the fold has been generated pnjfvemyfrtigbxm
2023-08-25 10:31:10,057:INFO:a random identifier for the fold has been generated dwhkfdecqdiyqjgz
2023-08-25 10:31:10,058:INFO:a random identifier for the fold has been generated skovwmefioiowpqr
2023-08-25 10:31:10,

In [5]:
sample = experiment.sample()

In [6]:
sample.calculate_scores()

{'acc': 0.49782800045327774,
 'sens': 0.4530337787907355,
 'spec': 0.5190310512995421,
 'bacc': 0.4860324150451388}

In [8]:
solve(experiment, sample.calculate_scores(), 1e-4).status == 1

2023-08-25 10:31:33,853:INFO:None: adding condition acc >= 0.49772800045327775
2023-08-25 10:31:33,854:INFO:None: adding condition acc <= 0.49792800045327773
2023-08-25 10:31:33,855:INFO:None: adding condition sens >= 0.4529337787907355
2023-08-25 10:31:33,856:INFO:None: adding condition sens <= 0.45313377879073546
2023-08-25 10:31:33,858:INFO:None: adding condition spec >= 0.5189310512995421
2023-08-25 10:31:33,859:INFO:None: adding condition spec <= 0.5191310512995421
2023-08-25 10:31:33,859:INFO:None: adding condition bacc >= 0.4859324150451388
2023-08-25 10:31:33,861:INFO:None: adding condition bacc <= 0.48613241504513877


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/afd64f03f3c74aec823bc06a495f6d49-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/afd64f03f3c74aec823bc06a495f6d49-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 354 RHS
At line 363 BOUNDS
At line 404 ENDATA
Problem MODEL has 8 rows, 40 columns and 240 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 214.611 - 0.00 seconds
Cgl0004I processed model has 4 rows, 26 columns (26 integer (0 of which binary)) and 78 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 215 found by DiveCoefficient after 9 iterations and 0 nodes (0.00 seconds)
Cbc0031I 4 added rows had average density of 17
Cbc0013I At root node, 4 cuts cha

True

In [4]:
dataset = Dataset(name='common_datasets.winequality-red-4', n_folds=2, n_repeats=3, aggregation='mor')

2023-08-24 21:54:05,538:INFO:querying p and n from looking up the dataset
2023-08-24 21:54:05,540:INFO:creating a folding based on the specification


In [5]:
ds2 = dataset.sample()

In [6]:
scores = ds2.calculate_scores()

In [7]:
res = solve(ds2, scores, 1e-4)

2023-08-24 21:54:05,573:INFO:common_datasets.winequality-red-4: adding condition acc >= 0.7552778160200251
2023-08-24 21:54:05,575:INFO:common_datasets.winequality-red-4: adding condition acc <= 0.7554778160200251
2023-08-24 21:54:05,576:INFO:common_datasets.winequality-red-4: adding condition sens >= 0.4379341880341881
2023-08-24 21:54:05,577:INFO:common_datasets.winequality-red-4: adding condition sens <= 0.4381341880341881
2023-08-24 21:54:05,577:INFO:common_datasets.winequality-red-4: adding condition spec >= 0.766178568348426
2023-08-24 21:54:05,578:INFO:common_datasets.winequality-red-4: adding condition spec <= 0.766378568348426
2023-08-24 21:54:05,580:INFO:common_datasets.winequality-red-4: adding condition bacc >= 0.6020563781913071
2023-08-24 21:54:05,582:INFO:common_datasets.winequality-red-4: adding condition bacc <= 0.6022563781913071


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/7447f4fdb1e84d6a82ad129ca633c5ea-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/7447f4fdb1e84d6a82ad129ca633c5ea-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 116 RHS
At line 125 BOUNDS
At line 138 ENDATA
Problem MODEL has 8 rows, 12 columns and 72 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 68.3177 - 0.00 seconds
Cgl0003I 0 fixed, 3 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 6 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 5 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 3 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 6 tightened b

In [8]:
res.status == 1

True

In [9]:
ds3 = dataset.populate(res)

['tp_winequality-red-4_0_0', 'tn_winequality-red-4_0_0']
[tn_winequality_red_4_0_0, tn_winequality_red_4_0_1, tn_winequality_red_4_1_0, tn_winequality_red_4_1_1, tn_winequality_red_4_2_0, tn_winequality_red_4_2_1, tp_winequality_red_4_0_0, tp_winequality_red_4_0_1, tp_winequality_red_4_1_0, tp_winequality_red_4_1_1, tp_winequality_red_4_2_0, tp_winequality_red_4_2_1]
{}
['tp_winequality-red-4_0_1', 'tn_winequality-red-4_0_1']
[tn_winequality_red_4_0_0, tn_winequality_red_4_0_1, tn_winequality_red_4_1_0, tn_winequality_red_4_1_1, tn_winequality_red_4_2_0, tn_winequality_red_4_2_1, tp_winequality_red_4_0_0, tp_winequality_red_4_0_1, tp_winequality_red_4_1_0, tp_winequality_red_4_1_1, tp_winequality_red_4_2_0, tp_winequality_red_4_2_1]
{}
['tp_winequality-red-4_1_0', 'tn_winequality-red-4_1_0']
[tn_winequality_red_4_0_0, tn_winequality_red_4_0_1, tn_winequality_red_4_1_0, tn_winequality_red_4_1_1, tn_winequality_red_4_2_0, tn_winequality_red_4_2_1, tp_winequality_red_4_0_0, tp_winequality

In [10]:
ds3.folds[0].variable_names

{'tp': 'tp_winequality-red-4_0_0', 'tn': 'tn_winequality-red-4_0_0'}

In [11]:
ds3.folds[0].populate(res)

['tp_winequality-red-4_0_0', 'tn_winequality-red-4_0_0']
[tn_winequality_red_4_0_0, tn_winequality_red_4_0_1, tn_winequality_red_4_1_0, tn_winequality_red_4_1_1, tn_winequality_red_4_2_0, tn_winequality_red_4_2_1, tp_winequality_red_4_0_0, tp_winequality_red_4_0_1, tp_winequality_red_4_1_0, tp_winequality_red_4_1_1, tp_winequality_red_4_2_0, tp_winequality_red_4_2_1]
{}


{'p': 27, 'n': 773, 'id': 'winequality-red-4_0_0', 'score_bounds': None, 'figures': {}}

In [2]:
Dataset(**generate_dataset_specification())

2023-08-24 21:29:42,241:INFO:querying p and n from looking up the dataset
2023-08-24 21:29:42,242:INFO:creating a folding based on the specification


{'id': 'lgwcnkmg', 'score_bounds': None, 'folds': [{'p': 7, 'n': 115, 'id': 'lgwcnkmg_0_0', 'score_bounds': None, 'figures': None}, {'p': 7, 'n': 115, 'id': 'lgwcnkmg_0_1', 'score_bounds': None, 'figures': None}, {'p': 7, 'n': 115, 'id': 'lgwcnkmg_0_2', 'score_bounds': None, 'figures': None}, {'p': 7, 'n': 115, 'id': 'lgwcnkmg_0_3', 'score_bounds': None, 'figures': None}, {'p': 7, 'n': 115, 'id': 'lgwcnkmg_0_4', 'score_bounds': None, 'figures': None}, {'p': 7, 'n': 114, 'id': 'lgwcnkmg_0_5', 'score_bounds': None, 'figures': None}, {'p': 7, 'n': 115, 'id': 'lgwcnkmg_1_0', 'score_bounds': None, 'figures': None}, {'p': 7, 'n': 115, 'id': 'lgwcnkmg_1_1', 'score_bounds': None, 'figures': None}, {'p': 7, 'n': 115, 'id': 'lgwcnkmg_1_2', 'score_bounds': None, 'figures': None}, {'p': 7, 'n': 115, 'id': 'lgwcnkmg_1_3', 'score_bounds': None, 'figures': None}, {'p': 7, 'n': 115, 'id': 'lgwcnkmg_1_4', 'score_bounds': None, 'figures': None}, {'p': 7, 'n': 114, 'id': 'lgwcnkmg_1_5', 'score_bounds': N

In [3]:
problem = Dataset(**generate_dataset_specification())

2023-08-24 21:29:56,703:INFO:querying p and n from looking up the dataset
2023-08-24 21:29:56,705:INFO:creating a folding based on the specification


In [4]:
sample = problem.sample()

In [5]:
sample.calculate_figures()

{'tp': 87, 'tn': 281, 'p': 128, 'n': 492}

In [6]:
sample.calculate_scores()

{'acc': 0.5935483870967742,
 'sens': 0.6796875,
 'spec': 0.5711382113821138,
 'bacc': 0.625412855691057}

In [11]:
res = solve(problem.set_fold_bounds(sample.get_fold_bounds(feasible=False)), sample.calculate_scores(), 1e-4)

2023-08-24 21:30:10,495:INFO:fold hepatitis_0_0: adding lower bound 0.0 for acc
2023-08-24 21:30:10,496:INFO:fold hepatitis_0_0: adding upper bound 0.9223076923076924 for acc
2023-08-24 21:30:10,497:INFO:fold hepatitis_0_0: adding lower bound 0.0 for sens
2023-08-24 21:30:10,498:INFO:fold hepatitis_0_0: adding upper bound 0.98 for sens
2023-08-24 21:30:10,499:INFO:fold hepatitis_0_0: adding lower bound 0.0 for spec
2023-08-24 21:30:10,500:INFO:fold hepatitis_0_0: adding upper bound 0.906829268292683 for spec
2023-08-24 21:30:10,501:INFO:fold hepatitis_0_0: adding lower bound 0.0 for bacc
2023-08-24 21:30:10,502:INFO:fold hepatitis_0_0: adding upper bound 0.9434146341463414 for bacc
2023-08-24 21:30:10,502:INFO:fold hepatitis_0_1: adding lower bound 0.0 for acc
2023-08-24 21:30:10,504:INFO:fold hepatitis_0_1: adding upper bound 0.49923076923076926 for acc
2023-08-24 21:30:10,508:INFO:fold hepatitis_0_1: adding lower bound 0.0 for sens
2023-08-24 21:30:10,509:INFO:fold hepatitis_0_1: add

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/d6364d6266ba4d36a63e238ae6a49be9-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/d6364d6266ba4d36a63e238ae6a49be9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 109 COLUMNS
At line 458 RHS
At line 563 BOUNDS
At line 588 ENDATA
Problem MODEL has 104 rows, 24 columns and 288 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 86.9872 - 0.00 seconds
Cgl0000I Cut generators found to be infeasible! (or unbounded)
Pre-processing says infeasible or unbounded
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



In [12]:
res.status == 1

False

In [9]:
sample.get_bounds()

{'acc': (0.5925483870967742, 0.5945483870967742),
 'sens': (0.6786875, 0.6806875),
 'spec': (0.5701382113821138, 0.5721382113821138),
 'bacc': (0.624412855691057, 0.626412855691057)}

In [10]:
problem.populate(res).calculate_scores()

{'acc': 0.5935483870967742,
 'sens': 0.6796875,
 'spec': 0.5711382113821138,
 'bacc': 0.625412855691057}

In [5]:
dataset.calculate_scores()

NameError: name 'dataset' is not defined

In [2]:
dataset = Dataset(p=5, n=10, n_folds=2, n_repeats=3, aggregation='mor')
dataset2 = Dataset(p=8, n=3, n_folds=2, n_repeats=3, aggregation='mor')
experiment = Experiment(datasets=[{'p': 5, 'n': 10, 'n_folds': 2, 'n_repeats': 3, 'aggregation': 'mor'},
                                    {'p': 8, 'n': 20, 'n_folds': 3, 'n_repeats': 3, 'aggregation': 'mor'}],
                        aggregation='mor')

2023-08-24 12:47:30,940:INFO:generating a random id for the dataset
2023-08-24 12:47:30,941:INFO:creating a folding based on the specification
2023-08-24 12:47:30,942:INFO:generating a random id for the dataset
2023-08-24 12:47:30,944:INFO:creating a folding based on the specification
2023-08-24 12:47:30,944:INFO:generating a random id for the dataset
2023-08-24 12:47:30,945:INFO:creating a folding based on the specification
2023-08-24 12:47:30,946:INFO:generating a random id for the dataset
2023-08-24 12:47:30,947:INFO:creating a folding based on the specification


In [3]:
experiment.sample()

{'id': None, 'aggregation': 'mor', 'datasets': [{'id': 'cninryupyaqqxvnf', 'score_bounds': None, 'tptn_bounds': None, 'folds': [{'p': 3, 'n': 5, 'id': 'cninryupyaqqxvnf_0_0', 'score_bounds': None, 'tptn_bounds': None, 'figures': {'tp': 1, 'tn': 4}, 'scores': {'acc': 0.625, 'sens': 0.3333333333333333, 'spec': 0.8, 'npv': 0.6666666666666666, 'ppv': 0.5, 'bacc': 0.5666666666666667, 'f1p': 0.4, 'fm': 0.4082482904638631, 'p': 3, 'n': 5, 'tp': 1, 'tn': 4}}, {'p': 2, 'n': 5, 'id': 'cninryupyaqqxvnf_0_1', 'score_bounds': None, 'tptn_bounds': None, 'figures': {'tp': 0, 'tn': 0}, 'scores': {'acc': 0.0, 'sens': 0.0, 'spec': 0.0, 'npv': 0.0, 'ppv': 0.0, 'bacc': 0.0, 'f1p': 0.0, 'fm': 0.0, 'p': 2, 'n': 5, 'tp': 0, 'tn': 0}}, {'p': 3, 'n': 5, 'id': 'cninryupyaqqxvnf_1_0', 'score_bounds': None, 'tptn_bounds': None, 'figures': {'tp': 2, 'tn': 0}, 'scores': {'acc': 0.25, 'sens': 0.6666666666666666, 'spec': 0.0, 'npv': 0.0, 'ppv': 0.2857142857142857, 'bacc': 0.3333333333333333, 'f1p': 0.4, 'fm': 0.43643

In [4]:
experiment.scores

{'acc': 0.4276895943562611,
 'sens': 0.4444444444444444,
 'spec': 0.43492063492063493,
 'bacc': 0.4396825396825397}

In [5]:
res = solve(experiment, experiment.scores, eps=1e-4)

2023-08-24 12:47:30,987:INFO:None: adding condition acc >= 0.4275895943562611
2023-08-24 12:47:30,989:INFO:None: adding condition acc <= 0.42778959435626107
2023-08-24 12:47:30,990:INFO:None: adding condition sens >= 0.44434444444444443
2023-08-24 12:47:30,992:INFO:None: adding condition sens <= 0.4445444444444444
2023-08-24 12:47:30,993:INFO:None: adding condition spec >= 0.43482063492063494
2023-08-24 12:47:30,993:INFO:None: adding condition spec <= 0.4350206349206349
2023-08-24 12:47:30,994:INFO:None: adding condition bacc >= 0.4395825396825397
2023-08-24 12:47:30,995:INFO:None: adding condition bacc <= 0.43978253968253966


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/a4ab4210a9304f23b32828ea9354afd9-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/a4ab4210a9304f23b32828ea9354afd9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 269 RHS
At line 278 BOUNDS
At line 309 ENDATA
Problem MODEL has 8 rows, 30 columns and 180 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 12.9964 - 0.00 seconds
Cgl0004I processed model has 4 rows, 10 columns (10 integer (0 of which binary)) and 30 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0031I 2 added rows had average density of 7.5
Cbc0013I At root node, 2 cuts changed objective from 12.9964 to 13 in 13 passes
Cbc0014I Cut generator 0 (Probing) - 0 row cuts average 

In [6]:
Experiment(**experiment.to_dict(problem_only=True)).populate(res).scores

{'acc': 0.4276785714285714,
 'sens': 0.4444444444444445,
 'spec': 0.43492063492063493,
 'bacc': 0.4396825396825397}

In [7]:
experiment.extract_bounds()

({'acc': [0.0, 0.8888888888888888],
  'sens': [0.0, 1.0],
  'spec': [0.0, 1.0],
  'bacc': [0.0, 0.9285714285714286]},
 {'tp': [0, 3], 'tn': [0, 6]})

In [8]:
experiment.check_bounds()

{'id': None,
 'figures': {'tp': 17, 'tn': 39, 'p': 39, 'n': 90},
 'scores': {'acc': 0.4276895943562611,
  'sens': 0.4444444444444444,
  'spec': 0.43492063492063493,
  'bacc': 0.4396825396825397},
 'all_bounds': False,
 'datasets': [{'id': 'cninryupyaqqxvnf',
   'figures': {'tp': 5, 'tn': 13, 'p': 15, 'n': 30},
   'scores': {'acc': 0.4047619047619048,
    'sens': 0.3333333333333333,
    'spec': 0.43333333333333335,
    'bacc': 0.3833333333333333},
   'score_bounds': None,
   'check_score_bounds': None,
   'tptn_bounds': None,
   'check_tptn_bounds': None,
   'all_bounds': False,
   'folds': [{'id': 'cninryupyaqqxvnf_0_0',
     'figures': {'tp': 1, 'tn': 4},
     'scores': {'acc': 0.625,
      'sens': 0.3333333333333333,
      'spec': 0.8,
      'npv': 0.6666666666666666,
      'ppv': 0.5,
      'bacc': 0.5666666666666667,
      'f1p': 0.4,
      'fm': 0.4082482904638631,
      'p': 3,
      'n': 5,
      'tp': 1,
      'tn': 4},
     'score_bounds': None,
     'check_score_bounds': None

In [9]:
experiment = Experiment(datasets=[{'name': 'common_datasets.ADA', 'aggregation': 'mor'},
                                    {'name': 'common_datasets.ecoli1', 'n_folds': 5, 'n_repeats': 3, 'aggregation': 'mor'}],
                        aggregation='mor')

2023-08-24 12:47:31,130:INFO:querying p and n from looking up the dataset
2023-08-24 12:47:31,132:INFO:creating a folding based on the specification
2023-08-24 12:47:31,133:INFO:querying p and n from looking up the dataset
2023-08-24 12:47:31,133:INFO:creating a folding based on the specification


In [10]:
experiment.sample()

{'id': None, 'aggregation': 'mor', 'datasets': [{'id': 'common_datasets.ADA', 'score_bounds': None, 'tptn_bounds': None, 'folds': [{'p': 1029, 'n': 3118, 'id': 'ADA_0_0', 'score_bounds': None, 'tptn_bounds': None, 'figures': {'tp': 703, 'tn': 833}, 'scores': {'acc': 0.370388232457198, 'sens': 0.6831875607385811, 'spec': 0.2671584348941629, 'npv': 0.7187230371009491, 'ppv': 0.2352744310575636, 'bacc': 0.475172997816372, 'f1p': 0.35001244709982576, 'fm': 0.40091964863096236, 'p': 1029, 'n': 3118, 'tp': 703, 'tn': 833}}], 'aggregation': 'mor', 'figures': {'tp': 703, 'tn': 833, 'p': 1029, 'n': 3118}, 'scores': {'acc': 0.370388232457198, 'sens': 0.6831875607385811, 'spec': 0.2671584348941629, 'bacc': 0.475172997816372}}, {'id': 'common_datasets.ecoli1', 'score_bounds': None, 'tptn_bounds': None, 'folds': [{'p': 16, 'n': 52, 'id': 'ecoli1_0_0', 'score_bounds': None, 'tptn_bounds': None, 'figures': {'tp': 13, 'tn': 11}, 'scores': {'acc': 0.35294117647058826, 'sens': 0.8125, 'spec': 0.21153846

In [11]:
experiment.calculate_scores()

{'acc': 0.4059506277884468,
 'sens': 0.6349271137026239,
 'spec': 0.33296333006648815,
 'bacc': 0.4839452218845561}

In [12]:
experiment.extract_bounds()

({'acc': [0.08955223880597014, 0.7352941176470589],
  'sens': [0.2, 1.0],
  'spec': [0.019230769230769232, 0.8846153846153846],
  'bacc': [0.1446078431372549, 0.8173076923076923]},
 {'tp': [3, 703], 'tn': [1, 833]})

In [13]:
scores = experiment.calculate_scores()

In [14]:
scores

{'acc': 0.4059506277884468,
 'sens': 0.6349271137026239,
 'spec': 0.33296333006648815,
 'bacc': 0.4839452218845561}

In [15]:
res = solve(experiment, scores, {'acc': 1e-4, 'sens': 1e-4, 'spec': 1e-4, 'bacc': 1e-4})

2023-08-24 12:47:31,212:INFO:None: adding condition acc >= 0.4058506277884468
2023-08-24 12:47:31,215:INFO:None: adding condition acc <= 0.4060506277884468
2023-08-24 12:47:31,216:INFO:None: adding condition sens >= 0.6348271137026239
2023-08-24 12:47:31,217:INFO:None: adding condition sens <= 0.6350271137026239
2023-08-24 12:47:31,219:INFO:None: adding condition spec >= 0.33286333006648816
2023-08-24 12:47:31,220:INFO:None: adding condition spec <= 0.33306333006648814
2023-08-24 12:47:31,221:INFO:None: adding condition bacc >= 0.4838452218845561
2023-08-24 12:47:31,223:INFO:None: adding condition bacc <= 0.48404522188455607


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/c71d90ef7145457d9c5f4034c61a7968-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/c71d90ef7145457d9c5f4034c61a7968-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 286 RHS
At line 295 BOUNDS
At line 328 ENDATA
Problem MODEL has 8 rows, 32 columns and 192 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 508.474 - 0.00 seconds
Cgl0004I processed model has 4 rows, 8 columns (8 integer (0 of which binary)) and 24 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 512 found by DiveCoefficient after 14 iterations and 0 nodes (0.00 seconds)
Cbc0031I 3 added rows had average density of 4.3333333
Cbc0013I At root node, 3 cu

In [16]:
experiment2 = Experiment(**experiment.to_dict(raw_problem=True))

TypeError: Experiment.to_dict() got an unexpected keyword argument 'raw_problem'

In [ ]:
experiment2.populate_with_solution(res)

In [ ]:
experiment2.calculate_scores()

{'acc': 0.39800535613256993,
 'sens': 0.7194298671849693,
 'spec': 0.2903846153846154,
 'bacc': 0.5049072412847924}

In [ ]:
experiment2.check_bounds()

{'figures': {'tp': 922.0, 'tn': 453.0, 'p': 1260, 'n': 3895},
 'scores': {'acc': 0.39800535613256993,
  'sens': 0.7194298671849693,
  'spec': 0.2903846153846154,
  'bacc': 0.5049072412847924},
 'downstream_bounds': True,
 'datasets': [{'figures': {'tp': 758.0, 'tn': 0.0, 'p': 1029, 'n': 3118},
   'scores': {'acc': 0.18278273450687244,
    'sens': 0.7366375121477162,
    'spec': 0.0,
    'bacc': 0.3683187560738581},
   'score_bounds': None,
   'check_score_bounds': True,
   'tptn_bounds': None,
   'check_tptn_bounds': True,
   'downstream_bounds': True,
   'folds': [{'figures': {'tn': 0.0, 'tp': 758.0},
     'scores': {'acc': 0.18278273450687244,
      'sens': 0.7366375121477162,
      'spec': 0.0,
      'npv': 0.0,
      'ppv': 0.195562435500516,
      'bacc': 0.3683187560738581,
      'f1p': 0.309072375127421,
      'fm': 0.379550557839991,
      'p': 1029,
      'n': 3118,
      'tn': 0.0,
      'tp': 758.0},
     'score_bounds': None,
     'check_score_bounds': True,
     'tptn_boun

In [ ]:
#import pulp as pl
#lp_program = pl.LpProblem('feasibility')

In [ ]:
#tp = pl.LpVariable('tp', 0, 5, pl.LpInteger)
#tn = pl.LpVariable('tn', 0, 10, pl.LpInteger)

In [ ]:
#lp_program += tp

In [ ]:
#lp_program

In [ ]:
#lp_program += tp <= 5

In [ ]:
#lp_program

In [ ]:
dataset = Dataset(name='common_datasets.ADA', n_repeats=2, n_folds=3, aggregation='rom')

2023-08-24 00:26:13,357:INFO:querying p and n from looking up the dataset
2023-08-24 00:26:13,358:INFO:creating a folding based on the specification


In [ ]:
dataset.simulate()

AttributeError: 'Dataset' object has no attribute 'simulate'

In [ ]:
dataset.calculate_scores()

{'acc': 0.4115022908126357,
 'sens': 0.3542274052478134,
 'spec': 0.4304041051956382,
 'bacc': 0.3923157552217258}

In [ ]:
dataset2 = Dataset(**dataset.to_dict())

In [ ]:
dataset.to_dict()

{'id': 'common_datasets.ADA',
 'score_bounds': None,
 'tptn_bounds': None,
 'folds': [{'p': 343,
   'n': 1040,
   'id': 'ADA_0_0',
   'score_bounds': None,
   'tptn_bounds': None,
   'evaluation': {'tp': 304.0, 'tn': 153.0},
   'scores': {'acc': 0.3304410701373825,
    'sens': 0.8862973760932945,
    'spec': 0.14711538461538462,
    'npv': 0.796875,
    'ppv': 0.25524769101595296,
    'bacc': 0.5167063803543396,
    'f1p': 0.3963494132985658,
    'fm': 0.4756315368027136,
    'p': 343,
    'n': 1040,
    'tp': 304.0,
    'tn': 153.0}},
  {'p': 343,
   'n': 1039,
   'id': 'ADA_0_1',
   'score_bounds': None,
   'tptn_bounds': None,
   'evaluation': {'tp': 0.0, 'tn': 679.0},
   'scores': {'acc': 0.4913169319826339,
    'sens': 0.0,
    'spec': 0.6535129932627527,
    'npv': 0.6643835616438356,
    'ppv': 0.0,
    'bacc': 0.32675649663137635,
    'f1p': 0.0,
    'fm': 0.0,
    'p': 343,
    'n': 1039,
    'tp': 0.0,
    'tn': 679.0}},
  {'p': 343,
   'n': 1039,
   'id': 'ADA_0_2',
   'scor

In [ ]:
dataset2.calculate_scores()

{'acc': 0.4115022908126357,
 'sens': 0.3542274052478134,
 'spec': 0.4304041051956382,
 'bacc': 0.3923157552217258}

In [ ]:
dataset2.aggregation = 'mor'

In [ ]:
dataset2.calculate_scores()

{'acc': 0.4115200984736789,
 'sens': 0.3542274052478134,
 'spec': 0.43045704696330295,
 'bacc': 0.3923422261055582}

In [ ]:
dataset2.scores

{'acc': 0.4115200984736789,
 'sens': 0.3542274052478134,
 'spec': 0.43045704696330295,
 'bacc': 0.3923422261055582}

In [ ]:
dataset2.linear_programming

In [ ]:
res = solve(dataset, dataset2.scores, eps={'acc': 1e-4, 'sens': 1e-4, 'spec': 1e-4, 'bacc': 1e-4})

aaa
bbb
{'acc': 0.4115200984736789, 'sens': 0.3542274052478134, 'spec': 0.43045704696330295, 'bacc': 0.3923422261055582}
{'acc': 0.0001, 'sens': 0.0001, 'spec': 0.0001, 'bacc': 0.0001}
{'tp': 1*tp_ADA_0_0 + 1*tp_ADA_0_1 + 1*tp_ADA_0_2 + 1*tp_ADA_1_0 + 1*tp_ADA_1_1 + 1*tp_ADA_1_2 + 0, 'tn': 1*tn_ADA_0_0 + 1*tn_ADA_0_1 + 1*tn_ADA_0_2 + 1*tn_ADA_1_0 + 1*tn_ADA_1_1 + 1*tn_ADA_1_2 + 0, 'p': 2058, 'n': 6236, 'acc': 0.00012056908608632747*tn_ADA_0_0 + 0.00012056908608632747*tn_ADA_0_1 + 0.00012056908608632747*tn_ADA_0_2 + 0.00012056908608632747*tn_ADA_1_0 + 0.00012056908608632747*tn_ADA_1_1 + 0.00012056908608632747*tn_ADA_1_2 + 0.00012056908608632747*tp_ADA_0_0 + 0.00012056908608632747*tp_ADA_0_1 + 0.00012056908608632747*tp_ADA_0_2 + 0.00012056908608632747*tp_ADA_1_0 + 0.00012056908608632747*tp_ADA_1_1 + 0.00012056908608632747*tp_ADA_1_2 + 0.0, 'sens': 0.00048590864917395527*tp_ADA_0_0 + 0.00048590864917395527*tp_ADA_0_1 + 0.00048590864917395527*tp_ADA_0_2 + 0.00048590864917395527*tp_ADA_1_0 

In [ ]:
dataset2.populate_with_solution(res)

In [ ]:
dataset2.calculate_scores()

{'acc': 0.4114809454896286,
 'sens': 0.3542274052478134,
 'spec': 0.43038177735001604,
 'bacc': 0.3923045912989147}

In [ ]:
dataset.calculate_scores()

{'acc': 0.4115022908126357,
 'sens': 0.3542274052478134,
 'spec': 0.4304041051956382,
 'bacc': 0.3923157552217258}

In [ ]:
dataset.linear_programming

{'tp': 1*tp_ADA_0_0 + 1*tp_ADA_0_1 + 1*tp_ADA_0_2 + 1*tp_ADA_1_0 + 1*tp_ADA_1_1 + 1*tp_ADA_1_2 + 0,
 'tn': 1*tn_ADA_0_0 + 1*tn_ADA_0_1 + 1*tn_ADA_0_2 + 1*tn_ADA_1_0 + 1*tn_ADA_1_1 + 1*tn_ADA_1_2 + 0,
 'p': 2058,
 'n': 6236,
 'acc': 0.00012056908608632747*tn_ADA_0_0 + 0.00012056908608632747*tn_ADA_0_1 + 0.00012056908608632747*tn_ADA_0_2 + 0.00012056908608632747*tn_ADA_1_0 + 0.00012056908608632747*tn_ADA_1_1 + 0.00012056908608632747*tn_ADA_1_2 + 0.00012056908608632747*tp_ADA_0_0 + 0.00012056908608632747*tp_ADA_0_1 + 0.00012056908608632747*tp_ADA_0_2 + 0.00012056908608632747*tp_ADA_1_0 + 0.00012056908608632747*tp_ADA_1_1 + 0.00012056908608632747*tp_ADA_1_2 + 0.0,
 'sens': 0.00048590864917395527*tp_ADA_0_0 + 0.00048590864917395527*tp_ADA_0_1 + 0.00048590864917395527*tp_ADA_0_2 + 0.00048590864917395527*tp_ADA_1_0 + 0.00048590864917395527*tp_ADA_1_1 + 0.00048590864917395527*tp_ADA_1_2 + 0.0,
 'spec': 0.00016035920461834508*tn_ADA_0_0 + 0.00016035920461834508*tn_ADA_0_1 + 0.000160359204618345

In [ ]:
dataset.to_dict()

{'id': 'common_datasets.ADA',
 'score_bounds': None,
 'tptn_bounds': None,
 'folds': [{'p': 343,
   'n': 1040,
   'id': 'ADA_0_0',
   'score_bounds': None,
   'tptn_bounds': None,
   'evaluation': {'tp': 304.0, 'tn': 153.0},
   'scores': {'acc': 0.3304410701373825,
    'sens': 0.8862973760932945,
    'spec': 0.14711538461538462,
    'npv': 0.796875,
    'ppv': 0.25524769101595296,
    'bacc': 0.5167063803543396,
    'f1p': 0.3963494132985658,
    'fm': 0.4756315368027136,
    'p': 343,
    'n': 1040,
    'tp': 304.0,
    'tn': 153.0}},
  {'p': 343,
   'n': 1039,
   'id': 'ADA_0_1',
   'score_bounds': None,
   'tptn_bounds': None,
   'evaluation': {'tp': 0.0, 'tn': 679.0},
   'scores': {'acc': 0.4913169319826339,
    'sens': 0.0,
    'spec': 0.6535129932627527,
    'npv': 0.6643835616438356,
    'ppv': 0.0,
    'bacc': 0.32675649663137635,
    'f1p': 0.0,
    'fm': 0.0,
    'p': 343,
    'n': 1039,
    'tp': 0.0,
    'tn': 679.0}},
  {'p': 343,
   'n': 1039,
   'id': 'ADA_0_2',
   'scor

In [ ]:
dataset.calculate_scores()

{'acc': 0.4115022908126357,
 'sens': 0.3542274052478134,
 'spec': 0.4304041051956382,
 'bacc': 0.3923157552217258}

In [ ]:
fold = Fold(p=5, n=10, id='fold_0')

In [ ]:
fold2 = Fold(**fold.to_dict(True))

In [ ]:
fold2.to_dict()

{'p': 5,
 'n': 10,
 'id': 'fold_0',
 'score_bounds': None,
 'tptn_bounds': None,
 'evaluation': None,
 'scores': None}

In [ ]:
fold.simulate()

In [ ]:
fold.calculate_scores()

{'acc': 0.13333333333333333,
 'sens': 0.0,
 'spec': 0.2,
 'npv': 0.2857142857142857,
 'ppv': 0.0,
 'bacc': 0.1,
 'f1p': 0.0,
 'fm': 0.0,
 'p': 5,
 'n': 10,
 'tp': 0.0,
 'tn': 2.0}

In [ ]:
fold.scores

{'acc': 0.13333333333333333,
 'sens': 0.0,
 'spec': 0.2,
 'npv': 0.2857142857142857,
 'ppv': 0.0,
 'bacc': 0.1,
 'f1p': 0.0,
 'fm': 0.0,
 'p': 5,
 'n': 10,
 'tp': 0.0,
 'tn': 2.0}

In [ ]:
fold.evaluation

{'tp': 0.0, 'tn': 2.0}

In [ ]:
fold.scores

{'acc': 0.13333333333333333,
 'sens': 0.0,
 'spec': 0.2,
 'npv': 0.2857142857142857,
 'ppv': 0.0,
 'bacc': 0.1,
 'f1p': 0.0,
 'fm': 0.0,
 'p': 5,
 'n': 10,
 'tp': 0.0,
 'tn': 2.0}

In [ ]:
fold.p, fold.n

(5, 10)

In [ ]:
res = solve(fold, fold.scores, eps={'acc': 1e-4, 'sens': 1e-4, 'spec': 1e-4, 'bacc': 1e-4})

{'acc': 0.13333333333333333, 'sens': 0.0, 'spec': 0.2, 'npv': 0.2857142857142857, 'ppv': 0.0, 'bacc': 0.1, 'f1p': 0.0, 'fm': 0.0, 'p': 5, 'n': 10, 'tp': 0.0, 'tn': 2.0}
{'acc': 0.0001, 'sens': 0.0001, 'spec': 0.0001, 'bacc': 0.0001}
{'tp': tp_fold_0, 'tn': tn_fold_0, 'acc': 0.06666666666666667*tn_fold_0 + 0.06666666666666667*tp_fold_0 + 0.0, 'sens': 0.2*tp_fold_0 + 0.0, 'spec': 0.1*tn_fold_0 + 0.0, 'bacc': 0.05*tn_fold_0 + 0.1*tp_fold_0 + 0.0}
0.13333333333333333 0.0001 0.06666666666666667*tn_fold_0 + 0.06666666666666667*tp_fold_0
0.0 0.0001 0.2*tp_fold_0
0.2 0.0001 0.1*tn_fold_0
0.1 0.0001 0.05*tn_fold_0 + 0.1*tp_fold_0
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gykovacs/anaconda3/envs/mlscorecheck/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/b9b5d0b1b43b4522981de5ca07abd1ac-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/b9b5d0b1b43b4522981de5ca07abd1ac-pulp.sol (default strategy 1)
At line 2

In [ ]:
res.status

1

In [ ]:
res.variables()

[tn_fold_0, tp_fold_0]

In [ ]:
fold.populate_with_solution(res).calculate_scores()

{'acc': 0.13333333333333333,
 'sens': 0.0,
 'spec': 0.2,
 'npv': 0.2857142857142857,
 'ppv': 0.0,
 'bacc': 0.1,
 'f1p': 0.0,
 'fm': 0.0,
 'p': 5,
 'n': 10,
 'tn': 2.0,
 'tp': 0.0}

In [ ]:
fold.evaluation

{'tn': 2.0, 'tp': 0.0}

In [ ]:
fold.calculate_scores()

{'acc': 0.13333333333333333,
 'sens': 0.0,
 'spec': 0.2,
 'npv': 0.2857142857142857,
 'ppv': 0.0,
 'bacc': 0.1,
 'f1p': 0.0,
 'fm': 0.0,
 'p': 5,
 'n': 10,
 'tn': 2.0,
 'tp': 0.0}